In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from datetime import datetime, timezone
from merge_dataframes import merge_dataframes

In [2]:
def get_list(version):
    return pd.read_csv(
        Path.cwd().parent / 
        "Elementary flow lists" / 
        "outputs" / 
        f"ecoinvent-{version}" / 
        f"ecoinvent-{version}.csv"
    )

In [3]:
PAIRS = [
    ("3.6", "3.7"),
    ("3.7", "3.8"),
    ("3.8", "3.9"),
]

In [4]:
for source_version, target_version in PAIRS:
    print(source_version, target_version)
    source = get_list(source_version)
    target = get_list(target_version)
    source_list_name = f"ecoinvent-{source_version}.csv"
    target_list_name = f"ecoinvent-{target_version}.csv"

    output = Path.cwd() / "outputs" / f"ecoinvent-{source_version}-ecoinvent-{target_version}"
    output.mkdir(parents=True, exist_ok=True)
    
    df = merge_dataframes(
        get_list(source_version), 
        get_list(target_version), 
        source_list_name, 
        target_list_name, 
        "Chris Mutel"
    )
    df.to_csv(
        output / f"ecoinvent-{source_version}-ecoinvent-{target_version}.csv", 
        index=False
    )

3.6 3.7
3.7 3.8
3.8 3.9
Inconsistent units, fix manually


,SourceFlowName,SourceFlowUUID,SourceFlowContext,SourceUnit,TargetFlowName,TargetFlowUUID,TargetFlowContext,TargetUnit
1845,"Gas, mine, off-gas, process, coal mining",3ed5f377-344f-423a-b5ec-9a9a1162b944,natural resource|in ground,m3,"Gas, mine, off-gas, process, coal mining",3ed5f377-344f-423a-b5ec-9a9a1162b944,natural resource|in ground,Sm3
1846,"Gas, natural, in ground",7c337428-fb1b-45c7-bbb2-2ee4d29e17ba,natural resource|in ground,m3,"Gas, natural",7c337428-fb1b-45c7-bbb2-2ee4d29e17ba,natural resource|in ground,Sm3
